# #CAPSTONE PROJECT

### THE BATTLE OF NEIGHBORHOODS : Expats Location Engine

### Data Exploring

In [62]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
!pip install geocoder
import geocoder 
!pip install folium
import folium
import geopy
import tqdm
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [63]:
!wget -O Pennsylvania_Municipalities.csv https://opendata.socrata.com/api/views/vr4q-nrmd/rows.csv?accessType=DOWNLOAD
print("Data source downloaded")

--2019-07-27 20:59:29--  https://opendata.socrata.com/api/views/vr4q-nrmd/rows.csv?accessType=DOWNLOAD
Resolving opendata.socrata.com (opendata.socrata.com)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to opendata.socrata.com (opendata.socrata.com)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘Pennsylvania_Municipalities.csv’

    [ <=>                                   ] 43,151      --.-K/s   in 0.03s   

Last-modified header invalid -- time-stamp ignored.
2019-07-27 20:59:29 (1.29 MB/s) - ‘Pennsylvania_Municipalities.csv’ saved [43151]

Data source downloaded


In [64]:
df = pd.read_csv("Pennsylvania_Municipalities.csv")
df.head()

,Municipality,Liquor Ban,Beer Ban,Distributor Ban,State Store Ban
0,"Adams Township, Butler County",Allowed,1935,Allowed,Allowed
1,"Adams Township, Snyder County",1955,1955,Allowed,Allowed
2,"Akron Borough, Lancaster County",1953,Allowed,Allowed,Allowed
3,"Aldan Borough, Delaware County",1941,1941,Allowed,Allowed
4,"Alexandria Borough, Huntingdon County",Allowed,1934,Allowed,Allowed


In [65]:
penn_data = df[["Municipality"]]

In [66]:
penn_data.head()

,Municipality
0,"Adams Township, Butler County"
1,"Adams Township, Snyder County"
2,"Akron Borough, Lancaster County"
3,"Aldan Borough, Delaware County"
4,"Alexandria Borough, Huntingdon County"


In [67]:
penn_data[["Borough", "County"]] = penn_data['Municipality'].str.split(',',expand=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [68]:
penn_data

,Municipality,Borough,County
0,"Adams Township, Butler County",Adams Township,Butler County
1,"Adams Township, Snyder County",Adams Township,Snyder County
2,"Akron Borough, Lancaster County",Akron Borough,Lancaster County
3,"Aldan Borough, Delaware County",Aldan Borough,Delaware County
4,"Alexandria Borough, Huntingdon County",Alexandria Borough,Huntingdon County
5,"Allegheny Township, Butler County",Allegheny Township,Butler County
6,"Allegheny Township, Venango County",Allegheny Township,Venango County
7,"Anthony Township, Lycoming County",Anthony Township,Lycoming County
8,"Antrim Township, Franklin County",Antrim Township,Franklin County
9,"Ararat Township, Susquehanna County",Ararat Township,Susquehanna County


In [69]:
penn_data = penn_data[["Borough", "County"]]
penn_data.head()
len(mun_data)

684

Duplicates dropping

In [70]:
penn_data.drop_duplicates(subset=['Borough'], inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [71]:
penn_data.shape

(578, 2)

In [72]:
penn_data.describe()

,Borough,County
count,578,578
unique,578,55
top,Fulton Township,York County
freq,1,40


In [73]:
penn_data_grp = penn_data.groupby(['County'], sort=False)['Borough'].apply(','.join).reset_index()
penn_data_grp.head()

,County,Borough
0,Butler County,"Adams Township,Allegheny Township,Brady Townsh..."
1,Lancaster County,"Akron Borough,Caernarvon Township,Drumore Town..."
2,Delaware County,"Aldan Borough,Bethel Township,Brookhaven Borou..."
3,Huntingdon County,"Alexandria Borough,Barree Township,Cass Townsh..."
4,Lycoming County,"Anthony Township,Cogan House Township,Gamble T..."


In [74]:
penn_data_grp.describe()

,County,Borough
count,55,55
unique,55,55
top,Crawford County,"Hickory Township,Little Beaver Township,Neshan..."
freq,1,1


In [75]:
column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

In [76]:
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.



In [77]:
from geopy.exc import GeocoderTimedOut
from time import sleep
def do_geocode(address):
    geopy = Nominatim()
    try:
        return geopy.geocode(address)
    except GeocoderTimedOut:
        sleep(1)
        return do_geocode(address)


for index, row in penn_data.iterrows():
    try:
        #address_1 = row['Borough'] 
        #address_2 = address_1.split(',')[-1]
        #address = address_2+","+"Pennsylvania,USA"
        address_1 = row['Borough'] 
        address = address_1+","+row['County']+","+"Pennsylvania"+","+"USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        #location = do_geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['District'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
        pass
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address = row['County']+","+"Pennsylvania,USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        #location = do_geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        #print(row['District'],address, latitude, longitude)
        n_hood
        pass

In [78]:
n_hood.head()

,Latitude,Longitude
0,40.717358,-80.012873
1,40.164064,-76.205099
2,39.919412,-75.400164
3,40.344633,-78.028119
4,41.138114,-79.738746


In [79]:
penn_data_geo = pd.concat([penn_data, n_hood[['Latitude', 'Longitude']]], axis=1)
penn_data_geo.shape

(657, 4)

In [80]:
penn_data_geo.head()

,Borough,County,Latitude,Longitude
0,Adams Township,Butler County,40.717358,-80.012873
1,NaN,NaN,40.164064,-76.205099
2,Akron Borough,Lancaster County,39.919412,-75.400164
3,Aldan Borough,Delaware County,40.344633,-78.028119
4,Alexandria Borough,Huntingdon County,41.138114,-79.738746


In [81]:
penn_data_geo = penn_data_geo.dropna(subset = ['Borough', 'County', 'Latitude', 'Longitude'])

In [82]:
penn_data_geo.head()

,Borough,County,Latitude,Longitude
0,Adams Township,Butler County,40.717358,-80.012873
2,Akron Borough,Lancaster County,39.919412,-75.400164
3,Aldan Borough,Delaware County,40.344633,-78.028119
4,Alexandria Borough,Huntingdon County,41.138114,-79.738746
5,Allegheny Township,Butler County,41.246568,-76.994516


In [83]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(penn_data_geo['County'].unique()),
        penn_data_geo.shape[0]
    )
)

The dataframe has 55 boroughs and 499 neighborhoods.


In [84]:
address = 'Pennsylvania,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pennsylvania are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pennsylvania are 40.9699889, -77.7278831.


In [85]:
map_penn = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(penn_data_geo['Latitude'], penn_data_geo['Longitude'], penn_data_geo['County'], penn_data_geo['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_penn) 
    
map_penn

In [87]:
penn_data_geo = penn_data_geo.reset_index(drop=True)
#penn_data_geo.columns = penn_data_geo.columns.str.lstrip()
penn_data_geo['County'] = penn_data_geo['County'].str.strip()

In [88]:
penn_data.describe()

,Borough,County
count,578,578
unique,578,55
top,Fulton Township,York County
freq,1,40


In [89]:
penn_data_new = penn_data_geo[penn_data_geo.County == 'Allegheny County'].reset_index(drop=True)
len(penn_data_new)
#columns = ["Borough", "County", "Latitude", "Longitude"]
#penn_data_new = penn_data_new.reindex(columns=columns)
penn_data_new.head()

,Borough,County,Latitude,Longitude
0,Bellevue Borough,Allegheny County,40.398036,-76.811517
1,Ben Avon Borough,Allegheny County,39.848541,-75.486389
2,Bradford Woods Borough,Allegheny County,39.876667,-75.391951
3,Edgewood Borough,Allegheny County,40.752960,-77.928250
4,Forest Hills Borough,Allegheny County,40.691662,-80.371000


In [90]:
address = 'Allegheny County, Pennsylvania'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Allegheny County are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Allegheny County are 40.4597204, -79.9760405.


In [91]:
# create map of Manhattan using latitude and longitude values
map_allegheny = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(penn_data_new['Latitude'], penn_data_new['Longitude'], penn_data_new['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_allegheny)  
    
map_allegheny

In [92]:
penn_data_new.loc[2, 'Borough']

'Bradford Woods Borough'

In [93]:
neighborhood_latitude = penn_data_new.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = penn_data_new.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = penn_data_new.loc[2, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bradford Woods Borough are 39.8766669, -75.3919513045786.


In [94]:
CLIENT_ID = 'HDD0TFIAZXBAGTHQFUMMFD1TXISGGMXQPMJNNYE1SIT5Q1FO' # your Foursquare ID
CLIENT_SECRET = 'T5ZUW2WMHRJ2JDWKN4TOX1JITUJQ4UFOQV4E1U3M0GFJ1HVJ' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HDD0TFIAZXBAGTHQFUMMFD1TXISGGMXQPMJNNYE1SIT5Q1FO
CLIENT_SECRET:T5ZUW2WMHRJ2JDWKN4TOX1JITUJQ4UFOQV4E1U3M0GFJ1HVJ


In [116]:
radius = 1000
LIMIT = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HDD0TFIAZXBAGTHQFUMMFD1TXISGGMXQPMJNNYE1SIT5Q1FO&client_secret=T5ZUW2WMHRJ2JDWKN4TOX1JITUJQ4UFOQV4E1U3M0GFJ1HVJ&v=20180924&ll=39.8766669,-75.3919513045786&radius=1000&limit=500'

In [117]:
import requests
import json
from pandas.io.json import json_normalize

In [119]:
results = requests.get(url).json()

In [120]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [121]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
len(nearby_venues)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
len(nearby_venues)
print('{} venues by Foursquare.'.format(nearby_venues.shape[0]))

45 venues by Foursquare.


In [122]:
#Call this function on each neighborhood and create a new dataframe called allegheny_venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #print(nearby_venues)
    return(nearby_venues)

In [127]:
allegheny_venues = getNearbyVenues(names=penn_data_new['Borough'],
                                   latitudes=penn_data_new['Latitude'],
                                   longitudes=penn_data_new['Longitude']
                                  )

Bellevue Borough
Ben Avon Borough
Bradford Woods Borough
Edgewood Borough
Forest Hills Borough
Ingram Borough


In [128]:
allegheny_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ben Avon Borough,39.848541,-75.486389,Bethel Community Park,39.846563,-75.482629,Park
1,Bradford Woods Borough,39.876667,-75.391951,Lowe's Home Improvement,39.879045,-75.394831,Hardware Store
2,Bradford Woods Borough,39.876667,-75.391951,Wawa,39.876445,-75.393380,Convenience Store
3,Bradford Woods Borough,39.876667,-75.391951,Route 352 Diner,39.874053,-75.390347,Diner
4,Bradford Woods Borough,39.876667,-75.391951,Starbucks,39.876556,-75.395671,Coffee Shop


In [129]:
print('There are {} uniques categories.'.format(len(allegheny_venues['Venue Category'].unique())))
allegheny_venues.groupby('Neighborhood').count()

There are 24 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ben Avon Borough,1,1,1,1,1,1
Bradford Woods Borough,26,26,26,26,26,26
Forest Hills Borough,1,1,1,1,1,1
Ingram Borough,2,2,2,2,2,2


In [130]:
# one hot encoding
allegheny_venues_onehot = pd.get_dummies(allegheny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
allegheny_venues_onehot['Neighborhood'] = allegheny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [allegheny_venues_onehot.columns[-1]] + list(allegheny_venues_onehot.columns[:-1])
allegheny_venues_onehot = allegheny_venues_onehot[fixed_columns]

allegheny_venues_onehot.head()

,Neighborhood,Antique Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega,Diner,...,Italian Restaurant,Liquor Store,Mobile Phone Shop,Optical Shop,Park,Pet Store,Pizza Place,Salon / Barbershop,Shopping Plaza,Supermarket
0,Ben Avon Borough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Bradford Woods Borough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bradford Woods Borough,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bradford Woods Borough,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Bradford Woods Borough,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
allegheny_venues_onehot.shape

(30, 25)

In [132]:
allegheny_venues_grouped = allegheny_venues_onehot.groupby('Neighborhood').mean().reset_index()
allegheny_venues_grouped

,Neighborhood,Antique Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega,Diner,...,Italian Restaurant,Liquor Store,Mobile Phone Shop,Optical Shop,Park,Pet Store,Pizza Place,Salon / Barbershop,Shopping Plaza,Supermarket
0,Ben Avon Borough,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bradford Woods Borough,0.0,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.0,0.076923,...,0.038462,0.115385,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462
2,Forest Hills Borough,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ingram Borough,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000


In [52]:
num_top_venues = 5

for hood in allegheny_venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = allegheny_venues_grouped[allegheny_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ben Avon Borough----
                    venue  freq
0          Discount Store   0.5
1  Thrift / Vintage Store   0.5
2              Campground   0.0
3              Kids Store   0.0


----Bradford Woods Borough----
                    venue  freq
0              Campground   1.0
1          Discount Store   0.0
2              Kids Store   0.0
3  Thrift / Vintage Store   0.0


----Forest Hills Borough----
                    venue  freq
0              Kids Store   1.0
1              Campground   0.0
2          Discount Store   0.0
3  Thrift / Vintage Store   0.0




In [133]:
#Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [134]:
#function to display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = allegheny_venues_grouped['Neighborhood']

for ind in np.arange(allegheny_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(allegheny_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
len(neighborhoods_venues_sorted)

4

In [135]:
allegheny_venues_grouped.head()

,Neighborhood,Antique Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega,Diner,...,Italian Restaurant,Liquor Store,Mobile Phone Shop,Optical Shop,Park,Pet Store,Pizza Place,Salon / Barbershop,Shopping Plaza,Supermarket
0,Ben Avon Borough,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bradford Woods Borough,0.0,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.0,0.076923,...,0.038462,0.115385,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462
2,Forest Hills Borough,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ingram Borough,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000


### Neighbourhoods Clustering

In [136]:
# set number of clusters
kclusters = 4

allegheny_grouped_clustering = allegheny_venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(allegheny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 2, 1], dtype=int32)

In [137]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ben Avon Borough,Park,Supermarket,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega
1,Bradford Woods Borough,Liquor Store,Salon / Barbershop,Diner,Supermarket,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store
2,Forest Hills Borough,Deli / Bodega,Supermarket,Shopping Plaza,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Diner
3,Ingram Borough,Antique Shop,Pizza Place,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega


In [138]:
penn_data_new

,Borough,County,Latitude,Longitude
0,Bellevue Borough,Allegheny County,40.398036,-76.811517
1,Ben Avon Borough,Allegheny County,39.848541,-75.486389
2,Bradford Woods Borough,Allegheny County,39.876667,-75.391951
3,Edgewood Borough,Allegheny County,40.752960,-77.928250
4,Forest Hills Borough,Allegheny County,40.691662,-80.371000
5,Ingram Borough,Allegheny County,39.778995,-76.116618


In [139]:
penn_data_new_1 = penn_data_new.iloc[[1,2,4,5],:]

In [140]:
penn_data_merged = penn_data_new_1

# add clustering labels
penn_data_merged['Cluster Labels'] = kmeans.labels_

# merge penn_grouped with penn_data to add latitude/longitude for each neighborhood
penn_data_merged = penn_data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

penn_data_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ben Avon Borough,Allegheny County,39.848541,-75.486389,0,Park,Supermarket,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega
2,Bradford Woods Borough,Allegheny County,39.876667,-75.391951,3,Liquor Store,Salon / Barbershop,Diner,Supermarket,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store
4,Forest Hills Borough,Allegheny County,40.691662,-80.371000,2,Deli / Bodega,Supermarket,Shopping Plaza,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Diner
5,Ingram Borough,Allegheny County,39.778995,-76.116618,1,Antique Shop,Pizza Place,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega


In [141]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(penn_data_merged['Latitude'].round(4), penn_data_merged['Longitude'].round(4), penn_data_merged['Borough'], penn_data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clustering

#### Cluster 1

In [142]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 0, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ben Avon Borough,Park,Supermarket,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega


#### Cluster 2

In [143]:

penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 1, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Ingram Borough,Antique Shop,Pizza Place,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Deli / Bodega


#### Cluster 3

In [144]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 2, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Forest Hills Borough,Deli / Bodega,Supermarket,Shopping Plaza,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Credit Union,Diner


#### Cluster 4

In [145]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 3, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bradford Woods Borough,Liquor Store,Salon / Barbershop,Diner,Supermarket,Food & Drink Shop,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store
